# Decision Tree
In this document, we implement the decision tree algorithm (machine learning model) from scratch in Python. To this end, we use the guidelines provided by AssemblyAI in https://www.youtube.com/watch?v=NxEHSAfFlK8.

It is worth mentioning that, we use Information Gain (IG) as the metric to split data.

## Import Libraries

In [1]:
import numpy as np
from collections import Counter

For a tree, two classes are required. One is the <em>Node</em> class to create the nodes in the tree, and the other is to form the tree itself based on
the features and the splitting criteria (here is IG).

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature   = feature     # feature to split
        self.threshold = threshold   # threshold to split
        self.left      = left        # left child
        self.right     = right       # right child
        self.value     = value       # value

    def is_leaf_node(self):
        # return self.value is not None
        return self.left is None and self.right is None

In [3]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split     # stopping criterion: minimum number of samples in a leaf node
        self.max_depth         = max_depth             # stopping criterion: maximum depth of the tree
        self.n_features        = n_features            # number of features
        self.root              = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._build_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    # auxiliary functions
    # build the tree
    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels              = len(np.unique(y))

        # check stopping criteria: min_samples_split, max_depth, n_labels            
        if (n_samples < self.min_samples_split or depth >= self.max_depth or n_labels == 1):
            counter = Counter(y)
            leaf_value = counter.most_common(1)[0][0]
            return Node(value=leaf_value)

        # find the best split
        feature_indices = np.random.choice(n_features, self.n_features, replace=False)   # randomly selects a set of features (self.n_features) from n_features
        best_feature, best_threshold = self._find_best_split(X, y, feature_indices)

        # create child nodes
        left_index, right_index = self._split(X[:, best_feature], best_threshold)
        left = self._build_tree(X[left_index, :], y[left_index], depth + 1)
        right = self._build_tree(X[right_index, :], y[right_index], depth + 1)

        return Node(best_feature, best_threshold, left, right)


    # find the best feature and threshold for splitting
    def _find_best_split(self, X, y, feature_indices):
        best_gain = -1
        split_index, split_threshold = None, None
        for feature_index in feature_indices:
            X_col = X[:, feature_index]
            thresholds = np.unique(X_col)
            for threshold in thresholds:
                # calculate IG
                gain = self._information_gain(X_col, y, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_index = feature_index
                    split_threshold = threshold

        return split_index, split_threshold

    # calculate information gain
    def _information_gain(self, X, y, threshold):
        # parent entropy: entropy before splitting (w.r.t. labels)
        H_parent = self._entropy(y)

        # children entropy: entropy after splitting based on the threshold
        # create children
        left_indices, right_indices = self._split(X, threshold)
        n_left, n_right             = len(left_indices), len(right_indices)
        
        if n_left == 0 or n_right == 0:
            return 0              # IG = 0

        # calculate the weighted average entropy of children
        n = len(y)
        H_left, H_right = self._entropy(y[left_indices]), self._entropy(y[right_indices])
        H_children = (n_left/n) * H_left + (n_right/n) * H_right

        # calculate the IG
        IG = H_parent - H_children
        return IG

    # form children
    def _split(self, X, threshold):
        left_index  = np.argwhere(X <= threshold).flatten()
        right_index = np.argwhere(X  > threshold).flatten()
        return left_index, right_index

    # calculate entropy
    def _entropy(self, y):
        hist = np.bincount(y)
        probabilities = hist / len(y)
        return -np.sum([p * np.log(p) for p in probabilities if p > 0])

    # traverse tree for predictions
    def _traverse_tree(self, X, node):
        if node.is_leaf_node():
            return node.value

        if X[node.feature] <= node.threshold:
            return self._traverse_tree(X, node.left)

        return self._traverse_tree(X, node.right)

## Fit Model

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [5]:
dataset = datasets.load_breast_cancer()
X, y = dataset.data, dataset.target

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
clf = DecisionTree()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f"Accuracy: {accuracy*100:.2f}")

Accuracy: 93.86
